In [2]:
from mxnet import nd
import random
import zipfile
import collections
import d2lzh as d2l
import math
from mxnet import autograd,gluon,nd
from mxnet.gluon import data as gdata,loss as gloss,nn,utils as gutils
import os
import tarfile
import random
import sys
import time
import zipfile
import numpy as np
from mxnet import init
from mxnet.contrib import text

In [3]:
def read_embedding_params(filePath):
    idx_to_token=np.load(filePath+'idx_to_token.npy')
    #token_to_idx=np.load(filePath+'token_to_idx.npy')
    token_to_idx={tk:idx for idx,tk in enumerate(idx_to_token)}
    embedding_weight=nd.load(filePath+'embedding_weight.txt')
    return idx_to_token,token_to_idx,embedding_weight

In [4]:
idx_to_token,token_to_idx,embedding_weight=read_embedding_params("data/params/word2Vec/")
embedding_dim=20

In [5]:
def read_params(filePath):
     layer_one_W=nd.load(filePath+'conv_layer_one_W.txt')
     layer_one_B=nd.load(filePath+'conv_layer_one_B.txt')
     layer_two_W=nd.load(filePath+'conv_layer_two_W.txt')
     layer_two_B=nd.load(filePath+'conv_layer_two_B.txt')
     layer_three_W=nd.load(filePath+'conv_layer_three_W.txt')
     layer_three_B=nd.load(filePath+'conv_layer_three_B.txt')
     layer_four_W=nd.load(filePath+'conv_layer_four_W.txt')
     layer_four_B=nd.load(filePath+'conv_layer_four_B.txt')
     layer_connect_W=nd.load(filePath+'ful_layer_connect_W.txt')
     layer_connect_B=nd.load(filePath+'ful_layer_connect_B.txt')
     params=[layer_one_W,layer_one_B,layer_two_W,layer_two_B,layer_three_W,layer_three_B,layer_four_W,layer_four_B,layer_connect_W,layer_connect_B]
     return params

In [6]:
text_size=800
embed_size=20

In [7]:
def conv_activation(w,b,z):
    sum=0;
    for t in z:
        sum+=t.sum()
    if sum==0:
        return 0
    temp=nd.dot(w,z)
    sum_two=temp+b
    for i in range(sum_two.shape[0]):
        for j in range(sum_two.shape[1]):
            sum_two[i][j]=max(sum_two[i][j],0)
    return sum_two

In [8]:
def conv_layer(X,param_w,param_b):
    windows_size=3
    Y=nd.zeros((X.shape[0],windows_size,param_w.shape[1]))
    for i in range(X.shape[0]):
        if(i+windows_size>=X.shape[0]):
            break
        Y[i]=conv_activation(param_w,param_b,X[i:i+windows_size])
    return Y

In [9]:
def init_params():
    conv_one_W=nd.zeros((20,3))
    conv_one_W=nd.random.normal(scale=0.01,shape=(conv_one_W.shape))
    conv_one_W.reshape(20,3)

    conv_one_B=nd.zeros((3,3))
    conv_one_B=nd.random.normal(scale=0.01,shape=(conv_one_B.shape))
    conv_one_B.reshape((3,3))
    
    conv_two_W=nd.zeros((9,3))
    conv_two_W=nd.random.normal(scale=0.01,shape=(conv_two_W.shape))
    conv_two_W.reshape(9,3)

    conv_two_B=nd.zeros((3,3))
    conv_two_B=nd.random.normal(scale=0.01,shape=(conv_two_B.shape))

    conv_three_W=nd.zeros((9,3))
    conv_three_W=nd.random.normal(scale=0.01,shape=(conv_three_W.shape))

    conv_three_B=nd.zeros((3,3))
    conv_three_B=nd.random.normal(scale=0.01,shape=(conv_three_B.shape))

    conv_four_W=nd.zeros((9,1))
    conv_four_W=nd.random.normal(scale=0.01,shape=(conv_four_W.shape))

    conv_four_B=nd.zeros(1)
    conv_four_B=nd.random.normal(scale=0.01,shape=(conv_four_B.shape))

    ful_connect_W=nd.zeros((100,80))
    ful_connect_W=nd.random.normal(scale=0.01,shape=(ful_connect_W.shape))

    ful_connect_B=nd.zeros((80,1))
    ful_connect_B=nd.random.normal(scale=0.01,shape=(ful_connect_B.shape))
    params=[conv_one_W,conv_one_B,conv_two_W,conv_two_B,conv_three_W,conv_three_B,conv_four_W,conv_four_B,ful_connect_W,ful_connect_B]

In [10]:
def myMax(X_one,X_two):
    sum_one=X_one.sum()
    sum_two=X_two.sum()
    if sum_one>sum_two:
        return X_one
    else :
        return X_two

In [11]:
def max_pool(X):
    temp= int(X.shape[0]/2)
    Y=nd.zeros((temp,X.shape[1]))
    j=0
    for i in range(X.shape[0]-1):
        if i%2==0:
            Y[j]=myMax(X[i],X[i+1])
            j+=1
    return Y

In [12]:
def full_connect_layer(X,W,B):
    Y=nd.zeros(W.shape[1])
    Y=nd.dot(X.T,W).T+B
    return Y.tanh()


In [13]:
def save_params(params,filePath):
    layer_one_W,layer_one_B,layer_two_W,layer_two_B,layer_three_W,layer_three_B,layer_four_W,layer_four_B,layer_connect_W,layer_connect_B=params
    file=filePath
    nd.save(file+'conv_layer_one_W.txt',layer_one_W)
    nd.save(file+'conv_layer_one_B.txt',layer_one_B)
    nd.save(file+'conv_layer_two_W.txt',layer_two_W)
    nd.save(file+'conv_layer_two_B.txt',layer_two_B)
    nd.save(file+'conv_layer_three_W.txt',layer_three_W)
    nd.save(file+'conv_layer_three_B.txt',layer_three_B)
    nd.save(file+'conv_layer_four_W.txt',layer_four_W)
    nd.save(file+'conv_layer_four_B.txt',layer_four_B)
    nd.save(file+'ful_layer_connect_W.txt',layer_connect_W)
    nd.save(file+'ful_layer_connect_B.txt',layer_connect_B)
    return

In [15]:
params=read_params('data/params/conv/')

In [16]:
def dropout_layer(X,drop_prob):
    assert 0 <= drop_prob <=1
    keep_prob=1-drop_prob
    if keep_prob==0:
        return X.zeros_like()
    mask=nd.random.uniform(0,1,X.shape)<keep_prob
    return mask*X/keep_prob

In [111]:
class CNNCodeRecheckTrainModel(nn.Block):
    def __init__(self,vocab_size,embedding_weight,embed_size,num_channels,**kwargs):
        super(CNNCodeRecheckTrainModel,self).__init__(**kwargs)
        #参与训练嵌入层
        self.embedding=nn.Embedding(input_dim=vocab_size,output_dim=embed_size)
        #不参与训练嵌入层
        #self.constant_embedding=nn.Embedding(input_dim=vocab_size,output_dim=embed_size)
        #申请梯度
        params=read_params('data/params/conv/')
        for param in params:
            #print(param[0])
            param[0].attach_grad()
        conv_one_W,conv_one_B,conv_two_W,conv_two_B,conv_three_W,conv_three_B,conv_four_W,conv_four_B,full_connect_W,full_connect_B=params
        self.conv_one_W=conv_one_W
        self.conv_one_B=conv_one_B
        self.conv_two_W=conv_two_W
        self.conv_two_B=conv_two_B
        self.conv_three_W=conv_three_W
        self.conv_three_B=conv_three_B
        self.conv_four_W=conv_four_W
        self.conv_foru_B=conv_four_B
        self.full_connect_W=full_connect_W
        self.full_connect_B=full_connect_B
        self.conv_layer=conv_layer
        self.max_pool_layer=max_pool_layer
        self.full_connect_layer=full_connect_layer
        self.dropout_layer=dropout_layer
        self.num_channels=num_channels
    def set_embedding_weight(self,embedding_weight):
        self.embedding.weight.set_data(embedding_weight)
    def read_params(filePath):
         layer_one_W=nd.load(filePath+'conv_layer_one_W.txt')
         layer_one_B=nd.load(filePath+'conv_layer_one_B.txt')
         layer_two_W=nd.load(filePath+'conv_layer_two_W.txt')
         layer_two_B=nd.load(filePath+'conv_layer_two_B.txt')
         layer_three_W=nd.load(filePath+'conv_layer_three_W.txt')
         layer_three_B=nd.load(filePath+'conv_layer_three_B.txt')
         layer_four_W=nd.load(filePath+'conv_layer_four_W.txt')
         layer_four_B=nd.load(filePath+'conv_layer_four_B.txt')
         layer_connect_W=nd.load(filePath+'ful_layer_connect_W.txt')
         layer_connect_B=nd.load(filePath+'ful_layer_connect_B.txt')
         params=[layer_one_W,layer_one_B,layer_two_W,layer_two_B,layer_three_W,layer_three_B,layer_four_W,layer_four_B,layer_connect_W,layer_connect_B]
         return params
    
    def forward(self,inputs,mode='train'):
        inputs_embeddings=nd.array([])
        if mode=='train':
            inputs_embeddings=nd.stack(self.embedding(inputs[0]),self.embedding(inputs[1]),self.embedding(inputs[2]))
        else :
            inputs_embeddings=nd.stack(self.embedding(inputs[0]),self.embedding(inputs[1]))
        

In [31]:
def read_file_data(filePath,token_to_idx,text_size):
    f=open(filePath)
    lines=f.readlines()
    raw_dataset=[st.split() for st in lines]
    raw_dataset.remove([])
    wordNum=0;
    dataset_temp=[]
    for st in raw_dataset:
         for wt in st:
                wordNum+=1
    dataset=[[token_to_idx[tk] for tk in st if tk in token_to_idx] for st in raw_dataset]
    dict_size=text_size #文本大小
    dataset=np.array(dataset)
    temp_one=np.array([])
    for row in dataset:
        for col in row:
            temp_one=np.append(temp_one,np.array(col))
    zeros=np.zeros(dict_size-wordNum)
    temp_two=np.append(temp_one,zeros)
    temp_three=np.array(temp_two)
    #print(temp_three)
    temp_three=temp_three.reshape(dict_size,-1)
    return temp_three

In [ ]:
temp=read_file_data("data/input_data/positiveText/positiveText_cnn.lex",token_to_idx,text_size)

In [32]:
def read_mult_input_data(fileDir,token_to_idx,text_size):
    positive_text=read_file_data(fileDir+'positiveText/positiveText_cnn.lex',token_to_idx,text_size)
    original_text=read_file_data(fileDir+'originalText/originalText_cnn.lex',token_to_idx,text_size)
    negative_text=read_file_data(fileDir+'negativeText/negativeText_cnn.lex',token_to_idx,text_size)
    return original_text,positive_text,negative_text

In [36]:
original_text,positive_text,negative_text=read_mult_input_data("data/input_data/",token_to_idx,text_size)
original_text=nd.array(original_text)
positive_text=nd.array(positive_text)
negative_text=nd.array(negative_text)

In [38]:
text=nd.stack(original_text,positive_text,negative_text)
#embedding_weight=embedding_weight
embedding_weight=nd.array(embedding_weight[0])

In [39]:
print(embedding_weight.shape)

(80, 20)


In [113]:
net=CNNCodeRecheckTrainModel(len(idx_to_token),embedding_weight,embed_size,3,params)
net.embedding.weight.initialize(init=init.Xavier(),force_reinit=True)
net.set_embedding_weight(embedding_weight)

In [ ]:
print(text)
#print(net.embedding.weight.data())

In [ ]:
print(nd.stack(net.embedding(original_text),net.embedding(positive_text),net.embedding(negative_text)))

In [ ]:
Y=net(text)